In [136]:
#STREAMS2TABLE
#written by Zachary J.J. Roper
#z.roper@vanderbilt.edu, zach@roper.io

#A script for converting raw event codes and timestamps into a trial-item dataset

In [137]:
#This notebook requires the following modules
import os
import numpy
import csv

In [138]:
#READ IN EVENT CODES
#The prepared file, out.csv, is a 2 column x session length (based on total number of trials) consistinting of event 
#codes (EVENTS[]) and their paired timestamps(TIMES[]). The file streams2text.m was used to generate these data 
#from the TDT data dump.

In [139]:
with open('out.csv') as csvfile:
    data = csv.reader(csvfile, delimiter = ',')
    events = []
    times = []
    for row in data:
        event = row[0]
        events.append(event)
        time = row[1]
        times.append(time)
pairedlist = list(zip(events,times))

In [140]:
events.count('2756') #catch trials

4

In [141]:
trial = 0
triallist = [None]*len(pairedlist)
eclist = [None]*len(pairedlist)
for i, data in enumerate(pairedlist):
    if pairedlist[i][0] == '2301':
        trial += 1
    triallist[i] = trial  
    eclist[i] = i
uberlist = list(zip(triallist,eclist,pairedlist))

In [142]:
#uberlist

In [143]:
'2810' in uberlist[74][2]

False

In [144]:
RT = [None]*len(pairedlist)
RTbeg = 0*len(pairedlist)
RTend = 0*len(pairedlist)
for i, trial in enumerate(uberlist):
    #print(i,trial)
    if '2651' in uberlist[i][2]:
        RTbeg = float(uberlist[i][2][1])
        focusTrial = uberlist[i][0]
        for p, case in enumerate(uberlist):
            if '2810' in uberlist[p][2] and uberlist[i][2] == focusTrial:
                RTend = float(uberlist[p][2][1])
    RT[i] = RTend - RTbeg

In [145]:
#RT

In [146]:
#The following provides indexed lists for each of the individual event codes of interest 

In [147]:
TargetOnset = []
FixOnset = []
Saccade = []
for i, x in enumerate(events):
    #print(i,x)
    if x == '2301':
        FixOnset.append(i)
    if x == '2651':
        TargetOnset.append(i)
    if x == '2810':
        Saccade.append(i)

In [148]:
#float(times[74])-float(times[70])

In [149]:
#Saccade

In [150]:
#for i, x in enumerate(Saccade):
#    print(Saccade[i],TargetOnset[i])

In [151]:
TOtimes = []
for items in TargetOnset:
    TOtimes.append(times[items])

In [152]:
Trials = []
TimeTrials = []
for i, x in enumerate(FixOnset):
    if i < len(FixOnset)-1:
        a = FixOnset[i]
        b = FixOnset[i+1]-1
        Trials.append(events[a:b])
        TimeTrials.append(times[a:b])
    else:
        a = FixOnset[i]
        Trials.append(events[a:])
        TimeTrials.append(times[a:])

In [153]:
#print(TimeTrials[0])

In [154]:
path = os.getcwd()
csvfile = (path+"/table.csv")
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(Trials)

In [155]:
accuracy = [None]*len(Trials)
abort = [None]*len(Trials)
error = [None]*len(Trials)
catch = [None]*len(Trials)
correct = [None]*len(Trials)
tloc = [None]*len(Trials)
for i, trial in enumerate(Trials):
    if '2600' in trial:
        accuracy[i] = 1 #Correct Saccade (2600)
    else:
        accuracy[i] = 0  
    if '2651' in trial: #fixation was maintained sufficiently long enough for target onset (2651)
        abort[i] = 0
    else:
        abort[i] = 1
    if '887' in trial:
        error[i] = 1
    else:
        error[i] = 0
    if '2756' in trial:
        catch[i] = 1
    else:
        catch[i] = 0
    if '888' in trial:
        correct[i] = 1
    else:
        correct[i] = 0
    #if '5000' in trial:
    #    tloc[i] = 360
    if '5045' in trial:
        tloc[i] = 45
    elif '5090' in trial:
        tloc[i] = 90
    elif '5135' in trial:
        tloc[i] = 135
    elif '5180' in trial:
        tloc[i] = 180
    elif '5225' in trial:
        tloc[i] = 225
    elif '5270' in trial:
        tloc[i] = 270
    elif '5315' in trial:
        tloc[i] = 315
    else:
        tloc[i] = 0

In [156]:
new_dict = {}
for i, trial in enumerate(Trials):
    new_dict[i+1] = dict(zip(Trials[i],TimeTrials[i]))

In [157]:
RT = [0]*len(Trials)
focusTrial = 0
for i, trial in enumerate(Trials):
    if '2651' in trial and '2810' in trial: #and '2600' in trial:
        focusTrial = i + 1
        for p, case in enumerate(uberlist):
            if uberlist[p][0] == focusTrial:
                if '2810' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTend = float(uberlist[p][2][1])
                if '2651' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTbeg = float(uberlist[p][2][1])
            RT[i] = RTend - RTbeg

In [158]:
#RT

In [159]:
checksum = sum(error)+sum(abort)+sum(catch)

In [160]:
checksum

326

In [161]:
path = os.getcwd()
csvfile = (path+"/behaviorArray.csv")
trial = 0
triallist = [None]*len(abort)
with open(csvfile, "w") as output:
    for i,x in enumerate(abort):
        trial += 1
        triallist[i] = trial 
        print(i+1,abort[i],accuracy[i],error[i],correct[i],catch[i],RT[i],tloc[i])
    header = ['Trial', 'Abort', 'Accuracy', 'Error', 'Correct', 'Catch', 'RT', 'TarLoc']    
    dataset = list(zip(triallist,abort,accuracy,error,correct,catch,RT,tloc))
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(dataset)

1 1 0 0 0 0 0 90
2 1 0 0 0 0 0 0
3 1 0 0 0 0 0 0
4 1 0 0 0 0 0 135
5 1 0 0 0 0 0 180
6 0 0 0 1 0 218.02999999999884 135
7 0 0 0 1 0 202.01460000000225 270
8 1 0 0 0 0 0 135
9 0 1 0 1 0 198.00069999999687 135
10 1 0 0 0 0 0 45
11 0 0 0 1 0 226.01729999999952 0
12 0 1 0 1 0 179.97819999999774 90
13 0 1 0 1 0 167.97679999999673 90
14 0 1 0 1 0 151.9614999999976 90
15 0 1 0 1 0 169.98389999999927 0
16 1 0 0 0 0 0 225
17 1 0 0 0 0 0 180
18 0 0 0 1 0 150.03629999999976 270
19 0 0 0 1 0 265.95330000000104 180
20 0 1 0 1 0 211.96820000000298 315
21 0 0 0 1 0 200.00759999999718 45
22 1 0 0 0 0 0 135
23 0 1 0 1 0 191.97959999999148 270
24 0 1 0 1 0 154.00960000000487 45
25 0 1 0 1 0 167.9768999999942 90
26 0 1 0 1 0 161.99679999999353 315
27 0 1 0 1 0 169.9841000000015 45
28 0 1 0 1 0 203.98099999999977 90
29 0 1 0 1 0 155.97560000000522 90
30 1 0 0 0 0 0 315
31 0 1 0 1 0 198.04170000000158 0
32 1 0 0 0 0 0 0
33 0 1 0 1 0 154.0097999999998 180
34 0 1 0 1 0 139.9603000000061 180
35 0 1 0 1 0 177.

516 0 1 0 1 0 159.94879999989644 315
517 0 1 0 1 0 216.0230000000447 135
518 0 1 0 1 0 301.9979000000749 0
519 1 0 0 0 0 0 270
520 0 0 1 0 0 173.99809999996796 90
521 0 1 0 1 0 209.96100000012666 0
522 1 0 0 0 0 0 45
523 0 0 0 1 0 185.9993999998551 225
524 0 1 0 1 0 199.96659999992698 315
525 0 1 0 1 0 210.08370000007562 45
526 1 0 0 0 0 0 135
527 0 1 0 1 0 207.99470000015572 135
528 0 1 0 1 0 180.01940000010654 270
529 0 1 0 1 0 248.01279999990948 270
530 0 1 0 1 0 207.9946999999229 0
531 1 0 0 0 0 0 180
532 0 1 0 1 0 190.0134000000544 135
533 0 1 0 1 0 179.97809999994934 315
534 1 0 0 0 0 0 225
535 0 1 0 1 0 215.98199999984354 45
536 0 1 0 1 0 191.9794000000693 0
537 0 1 0 1 0 204.02170000015758 0
538 1 0 0 0 0 0 0
539 1 0 0 0 0 0 135
540 0 1 0 1 0 185.95830000005662 0
541 0 1 0 1 0 200.00769999995828 270
542 0 0 1 0 0 436.01940000010654 135
543 0 1 0 1 0 204.02190000005066 135
544 0 1 0 1 0 173.99809999996796 90
545 1 0 0 0 0 0 270
546 0 1 0 1 0 259.9732999999542 270
547 0 1 0 1 0 2

In [162]:
sum(catch)

4